In [101]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<h2>Get Top 10 Cryptos History</h2>

In [102]:
from coinMarketCap import getTopHistory

dfs = getTopHistory(limit=10, days=90)


<h2>Calc Statistics</h2>

In [103]:
statColumns = [
    'avg[(max - min)/(max + min)]',
    'avg[(max - open)/ open ]',
    'avg[(min - open)/ open ]',
    'std[(max - min)/(max + min)]',
    'std[(max - open)/ open ]',
    'std[(min - open)/ open ]',
]
cryptoStat = pd.DataFrame(columns=statColumns)

for coin in dfs:
    df = dfs[coin]
    data = []

    data.append(((df.high - df.low)/(df.high + df.low)).mean())
    data.append(((df.high - df.open)/df.open).mean())
    data.append(((df.low - df.open)/df.open).mean())

    data.append(((df.high - df.low)/(df.high + df.low)).std())
    data.append(((df.high - df.open)/df.open).std())
    data.append(((df.low - df.open)/df.open).std())

    cryptoStat = cryptoStat.append(pd.DataFrame([data],columns=statColumns,index=[coin]))

cryptoStat.transpose()

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
avg[(max - min)/(max + min)],0.024970,0.032614,0.032937,0.058887,0.000584,0.038802,0.041427,0.051649,0.044432,0.000627
avg[(max - open)/ open ],0.026545,0.033839,0.034705,0.072850,0.000611,0.041978,0.045052,0.058184,0.050087,0.000591
avg[(min - open)/ open ],-0.023365,-0.031254,-0.031009,-0.047459,-0.000557,-0.035769,-0.037816,-0.045619,-0.039699,-0.000662
std[(max - min)/(max + min)],0.012695,0.016823,0.018518,0.031750,0.000247,0.024141,0.028040,0.029079,0.029992,0.000685
std[(max - open)/ open ],0.022386,0.027525,0.029157,0.066990,0.000401,0.043138,0.046597,0.050718,0.059780,0.000375
std[(min - open)/ open ],0.025344,0.032294,0.034856,0.041228,0.000337,0.040828,0.046770,0.046952,0.042755,0.001360


<h2>Calc fraction of days</h2>

In [104]:
statColumns_2 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_2 = pd.DataFrame(columns=statColumns_2)

for coin in dfs:
    df = dfs[coin]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.10]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.15]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.20]) / len(c) * 100),1))
    
    cryptoStat_2 = cryptoStat_2.append(pd.DataFrame([data],columns=statColumns_2,index=[coin]))

cryptoStat_2.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
[max - min] / open > 5 %,40.4,62.9,58.4,94.4,0.0,68.5,71.9,89.9,73.0,0.0
[max - min] / open > 10 %,4.5,11.2,14.6,49.4,0.0,25.8,27.0,41.6,31.5,0.0
[max - min] / open > 15 %,1.1,1.1,1.1,22.5,0.0,7.9,7.9,13.5,9.0,0.0
[max - min] / open > 20 %,0.0,1.1,1.1,13.5,0.0,1.1,4.5,6.7,4.5,0.0


percentage of days

<h2>Count of days when min == open AND max == open</h2>

In [105]:
statColumns_3 = [
    'min == open',
    'max == open',
]
cryptoStat_3 = pd.DataFrame(columns=statColumns_3)

for coin in dfs:
    df = dfs[coin]
    data = []
    data.append(len(df[df['low'] == df['open']]))
    data.append(len(df[df['high'] == df['open']]))
    
    cryptoStat_3 = cryptoStat_3.append(pd.DataFrame([data],columns=statColumns_3,index=[coin]))

cryptoStat_3["sum"] = cryptoStat_3.sum(axis=1)
cryptoStat_3.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
min == open,0,0,1,3,0,0,1,0,0,0
max == open,1,1,0,1,0,2,1,2,0,1
sum,1,1,1,4,0,2,2,2,0,1


<h2>Fraction of days with Condition: min < open AND max > open</h2>

In [106]:
statColumns_4 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_4 = pd.DataFrame(columns=statColumns_4)

for coin in dfs:
    df = dfs[coin]
    total = len(df)
    df = df[(df['low'] < df['open']) & (df['high'] > df['open'])]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / total * 100),1))
    data.append(round((len(c[c >= 0.10]) / total * 100),1))
    data.append(round((len(c[c >= 0.15]) / total * 100),1))
    data.append(round((len(c[c >= 0.20]) / total * 100),1))
    
    cryptoStat_4 = cryptoStat_4.append(pd.DataFrame([data],columns=statColumns_4,index=[coin]))

cryptoStat_4.T


,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
[max - min] / open > 5 %,40.4,62.9,57.3,89.9,0.0,66.3,69.7,87.6,73.0,0.0
[max - min] / open > 10 %,4.5,11.2,14.6,48.3,0.0,25.8,27.0,41.6,31.5,0.0
[max - min] / open > 15 %,1.1,1.1,1.1,21.3,0.0,7.9,7.9,13.5,9.0,0.0
[max - min] / open > 20 %,0.0,1.1,1.1,13.5,0.0,1.1,4.5,6.7,4.5,0.0


<h2>Final Result for Comparsion</h2>

In [107]:
pd.DataFrame(
    cryptoStat_2.T.append(cryptoStat_4.T).values,
    columns=pd.Index(list(dfs.keys())),
    index=pd.MultiIndex.from_product([['All days', 'max > open AND min < open'], statColumns_2]))


BTC   ETH   BNB   SOL  \
All days                  [max - min] / open >  5 %  40.4  62.9  58.4  94.4   
                          [max - min] / open > 10 %   4.5  11.2  14.6  49.4   
                          [max - min] / open > 15 %   1.1   1.1   1.1  22.5   
                          [max - min] / open > 20 %   0.0   1.1   1.1  13.5   
max > open AND min < open [max - min] / open >  5 %  40.4  62.9  57.3  89.9   
                          [max - min] / open > 10 %   4.5  11.2  14.6  48.3   
                          [max - min] / open > 15 %   1.1   1.1   1.1  21.3   
                          [max - min] / open > 20 %   0.0   1.1   1.1  13.5   

                                                     USDT   ADA   XRP   DOT  \
All days                  [max - min] / open >  5 %   0.0  68.5  71.9  89.9   
                          [max - min] / open > 10 %   0.0  25.8  27.0  41.6   
                          [max - min] / open > 15 %   0.0   7.9   7.9  13.5   
                          [max - min] / open > 20 %   0.0   1.1   4.5   6.7   
max > open AND min < open [max - min] / open >  5 %   0.0  66.3  69.7  87.6   
                          [max - min] / open > 10 %   0.0  25.8  27.0  41.6   
                          [max - min] / open > 15 %   0.0   7.9   7.9  13.5   
                          [max - min] / open > 20 %   0.0   1.1   4.5   6.7   

                                                     DOGE  USDC  
All days                  [max - min] / open >  5 %  73.0   0.0  
                          [max - min] / open > 10 %  31.5   0.0  
                          [max - min] / open > 15 %   9.0   0.0  
                          [max - min] / open > 20 %   4.5   0.0  
max > open AND min < open [max - min] / open >  5 %  73.0   0.0  
                          [max - min] / open > 10 %  31.5   0.0  
                          [max - min] / open > 15 %   9.0   0.0  
                          [max - min] / open > 20 %   4.5   0.0